<a href="https://colab.research.google.com/github/kelreeeeey/Traveloka-Hotel-Rangkin-ML/blob/main/Model-Hotel%20Rangking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API setting

In [ ]:
%pip install -q tensorflow-recommenders
%pip install tensorflow==2.8.0

     |████████████████████████████████| 85 kB 3.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 668.3 MB 57.6 MB/s
     |████████████████████████████████| 462 kB 5.4 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [ ]:
import datetime
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
print(tf.__version__)

2.8.0


# Download the dataset

In [ ]:
rating_final_1 = ! gdown -- 12d0cDxLjhti-K1LuPoqiHd8DrZ8cEqKChotels = ! gdown -- 1vsgfrdsD2x5mQoZjrBf_PUQDHO_FbOqg

# Data Prep

## Reading the data

In [ ]:
df_ratings = pd.read_csv('/content/ratings-final_1.csv')
df_hotels = pd.read_csv('/content/hotels.csv',usecols=[0,1])

In [ ]:
df_ratings.head()

,user_id,rating,hotelId
0,26,5,49
1,2,3,82
2,15,3,100
3,21,5,22
4,31,5,88


In [ ]:
df_hotels.head()

,hotelId,name
0,1,Orchard Hotel Singapore
1,2,Sheraton Tower Singapore
2,3,M Social Singapore
3,4,Swissotel The Stamford
4,5,Crowne Plaza CHANGI AIRPORT


## Merging the data

In [ ]:
df_merged = df_ratings.merge(df_hotels, on='hotelId')
df_merged.head()

,user_id,rating,hotelId,name
0,26,5,49,Pan Pacific Singapore
1,2,3,82,Fragrance Hotel Balestier
2,15,3,100,Aliwal Park Hotel
3,21,5,22,The Fullerton Hotel Singapore
4,31,5,88,Grand Hyatt Singapore


## Assigning user_id as str

In [ ]:
df_merged.user_id = df_merged.user_id.astype(str)
df_merged.head(100)

,user_id,rating,hotelId,name
0,26,5,49,Pan Pacific Singapore
1,2,3,82,Fragrance Hotel Balestier
2,15,3,100,Aliwal Park Hotel
3,21,5,22,The Fullerton Hotel Singapore
4,31,5,88,Grand Hyatt Singapore
...,...,...,...,...
95,99,4,97,Circular House Capsule hotel
96,97,4,68,Fairmont Singapore
97,86,4,47,Holiday Inn SINGAPORE ORCHARD CITY CENTRE
98,71,5,4,Swissotel The Stamford


## Data preprocessing

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices({
    'user_id': df_merged.user_id.to_list(),
    'name': df_merged.name.to_list(),
    'rating': df_merged.rating.to_list()
})

In [ ]:
hotels = tf.data.Dataset.from_tensor_slices({
    'name': df_merged.name.to_list()
})

In [ ]:
ratings = ratings.map(lambda x: {
    'user_id': x['user_id'],
    'rating' : x['rating'],
    'name': x['name'],
})
list(ratings.take(5).as_numpy_iterator())

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


[{'name': b'Pan Pacific Singapore', 'rating': 5, 'user_id': b'26'},
 {'name': b'Fragrance Hotel  Balestier', 'rating': 3, 'user_id': b'2'},
 {'name': b'Aliwal Park Hotel', 'rating': 3, 'user_id': b'15'},
 {'name': b'The Fullerton Hotel Singapore', 'rating': 5, 'user_id': b'21'},
 {'name': b'Grand Hyatt Singapore ', 'rating': 5, 'user_id': b'31'}]

In [ ]:
hotels = hotels.map(lambda x: x['name'])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80)
test = shuffled.skip(80).take(20)

In [ ]:
hotels_name = ratings.batch(1_000).map(lambda x: x["name"])
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

unique_hotels_name = np.unique(np.concatenate(list(hotels_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [ ]:
embedding_dimension = 32

# Build Model

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
hotel_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_hotels_name, mask_token=None),
  tf.keras.layers.Embedding(len(unique_hotels_name) + 1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=hotels.batch(128).map(hotel_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class HotelsModel(tfrs.Model):

  def __init__(self, user_model, hotel_model):
    super().__init__()
    self.hotel_model: tf.keras.Model = hotel_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_hotel_embeddings = self.hotel_model(features["name"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_hotel_embeddings)

In [ ]:
model = HotelsModel(user_model, hotel_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100).batch(64).cache()
cached_test = test.batch(32).cache()

# Model Fitting & Evaluating

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, s

In [ ]:
model.evaluate(cached_test, return_dict=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the ver

{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.05000000074505806,
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.44999998807907104,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'loss': 59.92403030395508,
 'regularization_loss': 0,
 'total_loss': 59.92403030395508}

# Testing the model

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((hotels.batch(100), hotels.batch(100).map(model.hotel_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["46"]))
print(f"Recommendations for user 46: {titles[0, :10]}")

Recommendations for user 46: [b'Porcelain Hotel by JL Asia' b'The Westin Singapore '
 b'The Clan Hotel Singapore by Far East Hospitality'
 b'ibis budget Singapore Pearl' b'K Hotel Aliwal (Premier)'
 b'Grand Hyatt Singapore ' b'YWCA Fort Canning'
 b'Santa Grand Hotel East Coast' b'Hilton Garden Inn  Singapore Serangoon'
 b'Arton Boutique Hotel']


# Saving the model

In [ ]:
# Export the query model.
tf.saved_model.save(index, "exported-retrieval/123")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: exported-retrieval/123/assets


INFO:tensorflow:Assets written to: exported-retrieval/123/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("exported-retrieval/123")
tflite_model = converter.convert()
open("converted_retrieval_model.tflite", "wb").write(tflite_model)

32528

In [ ]:
# Zip the SavedModel folder for easier download
!zip -r exported-retrieval.zip exported-retrieval/ 

updating: exported-retrieval/ (stored 0%)
updating: exported-retrieval/123/ (stored 0%)
updating: exported-retrieval/123/variables/ (stored 0%)
updating: exported-retrieval/123/variables/variables.index (deflated 34%)
updating: exported-retrieval/123/variables/variables.data-00000-of-00001 (deflated 14%)
updating: exported-retrieval/123/saved_model.pb (deflated 87%)
updating: exported-retrieval/123/assets/ (stored 0%)
